In [ ]:
#ensembling
from mlxtend.classifier import StackingCVClassifier
import numpy as np
import pandas as pd
import sys, os, glob
import imp
import seaborn as sns
sys.path.append('./src/')
import common_utils,my_ensembler, feature_utils, defines, model_utils, my_bert
from sklearn.neighbors import KNeighborsClassifier
from sklearn_crfsuite import scorers, CRF
from sklearn.pipeline import Pipeline
import json
from operator import itemgetter
from sklearn_crfsuite.utils import flatten
from sklearn_crfsuite.metrics import sequence_accuracy_score
from sklearn.linear_model import LogisticRegression
import scipy
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import classes
import pickle
import matplotlib.pyplot as plt
import copy

In [ ]:
dir_name='reparse'
seq_len=15
step=15
cv_split_name='cv_splits_10'
merged_str='scaled.binary0.50_merged'
doc_indices=np.arange(1,81)
doc_indices
test_doc_num=8
seed=42

In [ ]:
dir_name='reparse'
docs_map_name = 'scaled.binary.5_stop_docs_map.json'
crf_params_name='.802_21.07_07:26_best_params.json'
ens_crf_db_name   = 'ensemble_crf_cv_db_all'
ens_lg_db_name = 'ensemble_lg_cv_db_all'
cv_crf_bert_name = 'cv_crf_bert_all'
per_par=True
seq_len=3
step=3

### Open BERT predictions

In [ ]:
i=0
splits_bert={}
for i in range(10):
    json_name="split_{}_bert_valid_dict".format(i)
    splits_bert[i]=common_utils.load_json(dir_name,json_name)

In [ ]:
data=splits_bert[i]['valid_loss']

In [ ]:
data

In [ ]:
bert_loss_df=pd.DataFrame()
for k,v in splits_bert.items():
#     print(k,v['valid_loss'])
    bert_loss_df[k]=v['valid_loss']
common_utils.save_db(bert_loss_df,dir_name,"bert.loss.per.fold",False)

In [ ]:
splits_bert[y]['valid_loss']

In [ ]:
import matplotlib.cm as cm
colors = cm.rainbow(np.linspace(0, 1, len(splits_bert.keys())))

In [ ]:
for y, c in zip(splits_bert.keys(), colors):
    data=splits_bert[y]['valid_loss']
    min_x = np.argmin(data)
    min_y = np.min(data)
    plt.plot(data,marker='o',color=c)#, color='magenta', marker='o',mfc='pink' ) #plot the data
#     plt.scatter(min_x, min_y,c='r', label='minimum',zorder =1)
plt.xticks(range(0,len(data), 1)) #set the tick frequency on x-axis
# plt.legend()
plt.ylabel('Validation loss across splits') #set the label for y axis
plt.xlabel('epoch index') #set the label for x-axis
plt.title('Validation loss') #set the title of the graph
plt.show() #display the graph

### Check stories

In [ ]:
imp.reload(classes)
reloaded_dataset=pickle.load( open( os.path.join(os.getcwd(), defines.PATH_TO_DFS, dir_name,"dataset_full.p"), "rb" ) )

In [ ]:
sent_label_db = common_utils.concat_dbs(dir_name,"sent_db")

In [ ]:
imp.reload(classes)
dataset=classes.Dataset("")
dataset.copy_attr(reloaded_dataset)

In [ ]:
sent_label_db = common_utils.concat_dbs(dir_name,"sent_db")

In [ ]:
nar_db['len']=nar_db['text'].apply(lambda x: len(x))
nar_db

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(nar_db[nar_db['text'].str.contains("טלו")])

In [ ]:
nar_db=dataset.get_nar_df()

In [ ]:
with pd.option_context('display.max_colwidth', None,'display.max_rows', None): 
    display(nar_db.query('len<300'))

In [ ]:
sent_db

### Check document statistic

In [ ]:
par_contain_nar=[]
for key,val in reloaded_dataset.doc_map.items():
    for par in var.par_list:
        if par.y == 'is_nar'
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
vars(reloaded_dataset.doc_map[1].par_list[0])

In [ ]:
sent_label_db

In [ ]:
sent_label_db

In [ ]:
np.std(par_len),np.std(sent_len)

In [ ]:
np.median(par_len),np.median(sent_len)

In [ ]:
for key,val in reloaded_dataset.doc_map.items():
    par_len.append(len(val.par_list))
    sent_len.append(len(val.sent_list))

In [ ]:
vars(reloaded_dataset.doc_map[1].sent_list[13])

### Check paragraph statistic

In [ ]:
reloaded_dataset.doc_map[1].par_list

In [ ]:
imp.reload(classes)
new_dataset=classes.Dataset(reloaded_dataset.path)

In [ ]:
for k,doc in reloaded_dataset.doc_map.items():
    new_dataset.doc_map[k]={}
    new_dataset.doc_map[k] = classes.Document(k,new_dataset.path)
    new_dataset.doc_map[k].__dict__ = doc.__dict__

In [ ]:
new_dataset.doc_map[1]

In [ ]:
new_dataset.pack_sent_per_paragraph()

In [ ]:
par_stat_db=pd.DataFrame()

for k,doc in new_dataset.doc_map.items():
    for j,par in doc.par_map.items():
        par_stat_db.iloc[k,'doc_idx']=k
        

### Inspect BERT performance

In [ ]:
cv_db_bert=common_utils.load_db(dir_name,"cv_bert_all")

In [ ]:
bert_scores, bert_dict = model_utils.get_report_from_splits(cv_db_bert,"bert")

In [ ]:
bert_dict

In [ ]:
colors=['r','b']
linestyle=['dashed','solid']
f1={}
f1['1']=[]
f1['0']=[]
for label in f1.keys():
    for key,val in bert_dict.items():
        f1[label].append(val[label]['f1-score'])

In [ ]:
avg={}
avg['1']={}
avg['0']={}
avg['weighted avg']={}
for label in avg.keys():
    avg[label]={}
    avg[label]['recall']=[]
    avg[label]['prec']=[]
    avg[label]['f1']=[]

    for key,val in bert_dict.items():
        avg[label]['f1'].append(val[label]['f1-score'])
        avg[label]['recall'].append(val[label]['recall'])
        avg[label]['prec'].append(val[label]['precision'])

In [ ]:
for label,metrics in avg.items():
    for metric,value in metrics.items():
        print("{:>15} {:>10} {:.3f}".format(label,metric,np.mean(value)))
    

In [ ]:
from matplotlib import pyplot as plt
for y, c, st in zip(f1.keys(), colors,linestyle):
    data=f1[y]
    min_x = np.argmin(data)
    min_y = np.min(data)
    plt.plot(data,marker='o',color=c,linestyle=st,label=y)#, color='magenta', marker='o',mfc='pink' ) #plot the data
#     plt.scatter(min_x, min_y,c='r', label='minimum',zorder =1)
plt.xticks(range(0,len(data), 1)) #set the tick frequency on x-axis
plt.legend()
plt.ylabel('f-1 score') #set the label for y axis
plt.xlabel('fold index') #set the label for x-axis
plt.title('f-1 score per label') #set the title of the graph
plt.show() #display the graph

In [ ]:
f1

In [ ]:
imp.reload(classes)
rprt=classes.MyReport()
avg=rprt.get_avg_scores(bert_dict,labels=['0','1'])
for l in ['0','1']:
    print("{} {}".format(l,avg[l]['avg']))

In [ ]:
avg

In [ ]:
np.argmin(avg['1']['f1']),np.min(avg['1']['f1'])

In [ ]:
np.argmax(avg['1']['f1']),np.max(avg['1']['f1'])

In [ ]:
(np.max(avg['1']['f1'])-np.min(avg['1']['f1']))/np.min(avg['1']['f1'])

In [ ]:
np.std(avg['1']['f1']),np.std(avg['0']['f1'])

In [ ]:
colors=['r','b']
linestyle=['dashed','solid']
f1={}
f1['1']=[]
f1['0']=[]
for label in f1.keys():
    for key,val in bert_dict.items():
        f1[label].append(val[label]['f1-score'])

In [ ]:
from matplotlib import pyplot as plt
for y, c, st in zip(f1.keys(), colors,linestyle):
    data=f1[y]
    min_x = np.argmin(data)
    min_y = np.min(data)
    plt.plot(data,marker='o',color=c,linestyle=st,label=y)#, color='magenta', marker='o',mfc='pink' ) #plot the data
#     plt.scatter(min_x, min_y,c='r', label='minimum',zorder =1)
plt.xticks(range(0,len(data), 1)) #set the tick frequency on x-axis
plt.legend()
plt.ylabel('f-1 score') #set the label for y axis
plt.xlabel('fold index') #set the label for x-axis
plt.title('f-1 score per label') #set the title of the graph
plt.show() #display the graph

### Inspect CRF performance

In [ ]:
cv_db_crf=common_utils.load_db(dir_name,"cv_crf_best_params.78")

In [ ]:
cv_db_crf

In [ ]:
crf_scores, crf_dict = model_utils.get_report_from_splits(cv_db_crf,"crf")

In [ ]:
imp.reload(classes)
rprt=classes.MyReport()
labels=['not_nar','is_nar','weighted avg']
avg=rprt.get_avg_scores(crf_dict,labels=labels)
for l in labels:
    print("\n{:>15}".format(l),end =' ')
    for k,v in avg[l]['avg'].items():
#         print(k)
        print("{:>8} {:>8.3f}".format(k,v),end=' ')

In [ ]:
imp.reload(common_utils)
crf_best_params=common_utils.load_pickle("","crf.rs.params.793")
crf_best_params

In [ ]:
avg['weighted avg']

In [ ]:
crf_dict

In [ ]:
np.argmin(avg[labels[1]]['f1']),np.min(avg[labels[1]]['f1'])

In [ ]:
np.argmax(avg[labels[1]]['f1']),np.max(avg[labels[1]]['f1'])

In [ ]:
f1_crf

In [ ]:
from matplotlib import pyplot as plt
for y, c, st in zip(f1_crf.keys(), colors,linestyle):
    data=f1_crf[y]
    min_x = np.argmin(data)
    min_y = np.min(data)
    plt.plot(data,marker='o',color=c,linestyle=st,label=y)#, color='magenta', marker='o',mfc='pink' ) #plot the data
#     plt.scatter(min_x, min_y,c='r', label='minimum',zorder =1)
plt.xticks(range(0,len(data), 1)) #set the tick frequency on x-axis
plt.legend()
plt.ylabel('f-1 score') #set the label for y axis
plt.xlabel('fold index') #set the label for x-axis
plt.title('f-1 score per label') #set the title of the graph
plt.show() #display the graph

In [ ]:
ensemble_cv_db

### Load original splits

In [ ]:
cv_splits_loaded=common_utils.load_json(dir_name,"cv_splits_10")
cv_splits_loaded

In [ ]:
for k,v in cv_splits_loaded.items():
    if (4 in v['test'] and 76 in v['test']):
        print ('both in test',k)
    if (4 in v['train'] and 76 in v['train']):
        print ('both in train',k)

### Merge new CRF and old BERT

In [ ]:
cv_db_crf=cv_db_crf.assign(sent_idx=cv_db_crf.groupby(['crf_group']).cumcount())
cv_db_bert=cv_db_bert.assign(sent_idx=cv_db_bert.groupby(['bert_group']).cumcount())

In [ ]:
merged_cv_db=pd.merge(cv_db_crf,cv_db_bert,left_on=['crf_group','sent_idx'],right_on=['bert_group','sent_idx'],validate='one_to_one')


In [ ]:
cv_db_crf

In [ ]:
ensemble_res=pd.DataFrame()
ensemble_cv_db=model_utils.prepared_cross_validate_ensemble(LogisticRegression(random_state=0),ensemble_res,merged_cv_db,cv_splits_loaded)

In [ ]:
cv_db_crfx

In [ ]:
ens_scores, ens_dict = model_utils.get_report_from_splits(ensemble_cv_db,"ens")

In [ ]:
xxens_scores, ens_dict = model_utils.get_report_from_splits(merged_cv_db,"ens")

In [ ]:
imp.reload(classes)
rprt=classes.MyReport()
labels=['0','1']
labels.append('weighted avg')
avg=rprt.get_avg_scores(ens_dict,labels=labels)
for l in labels:
    print("\n{:>15}".format(l),end =' ')
    for k,v in avg[l]['avg'].items():
#         print(k)
        print("{:>8} {:>8.3f}".format(k,v),end=' ')

In [ ]:
ensemble_cv_db